In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers

In [ ]:
import os
os.chdir("drive/My Drive/target-guided-sat-chatbot")

In [ ]:
import time
import datetime
import random
import numpy as np
import json
import pickle
from itertools import chain
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from transformers import AdamW, get_linear_schedule_with_warmup, get_polynomial_decay_schedule_with_warmup

In [ ]:
mname = "facebook/blenderbot-400M-distill"
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)

In [ ]:
special_tokens = {'additional_special_tokens': ["<keyword>"]}
tokenizer.add_special_tokens(special_tokens)
print(tokenizer.all_special_tokens) # --> ['<s>', '</s>', '<unk>', '<pad>', '<mask>', '<keyword>']
print(tokenizer.all_special_ids)    # --> [1, 2, 3, 0, 8008, 8009]

model.resize_token_embeddings(len(tokenizer))

Load raw data

In [ ]:
with open('data/train/concepts_nv.json') as f:
  train_data_json = [json.loads(row) for row in f]
print(f"train length: {len(train_data_json)}")

In [ ]:
with open('data/dev/concepts_nv.json') as f:
  validation_data_json = [json.loads(row) for row in f]
print(f"validation length: {len(validation_data_json)}")

Dataset Class

In [ ]:
class KeywordGenerationDataset(Dataset):

  def __init__(self, data_json, tokenizer):

    self.input_ids = []
    self.labels = []

    bos = tokenizer.bos_token
    eos = tokenizer.eos_token
    kw_token = tokenizer.additional_special_tokens[0]

    max_input_length = 128
    max_context_length = 5

    for data in tqdm(data_json):
      dialog = data['dialog']
      concepts = data['concepts']

      for idx in range(1, len(dialog)):

        keywords = concepts[idx]

        start_idx = max(0, idx-max_context_length+1)
        contexts = dialog[start_idx:idx]
        response = dialog[idx]

        encoded_contexts = [tokenizer.encode(bos + c + eos, add_special_tokens=False) for c in contexts]

        random.shuffle(keywords)
        keywords_with_special_tokens = kw_token + kw_token.join(keywords)
        encoded_keywords = tokenizer.encode(keywords_with_special_tokens, add_special_tokens=False)

        input_ids = encoded_keywords + list(chain.from_iterable(encoded_contexts))
        assert len(input_ids) <= max_input_length

        labels = tokenizer.encode(response)

        self.input_ids.append(input_ids)
        self.labels.append(labels)

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return {'input_ids' : self.input_ids[idx], 'labels' : self.labels[idx]}



In [ ]:
def collate_fn(batch):
  input_ids, labels = [], []
  eos_id = tokenizer.eos_token_id
  pad_id = tokenizer.pad_token_id
  for b in batch:
    input_ids.append(torch.LongTensor(b['input_ids']))
    labels.append(torch.LongTensor(b['labels']))

  input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=pad_id)
  labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

  return {'input_ids' : input_ids, 'labels' : labels}

In [ ]:
train_dataset = KeywordGenerationDataset(train_data_json, tokenizer)
validation_dataset = KeywordGenerationDataset(validation_data_json, tokenizer)
print(len(validation_dataset))

In [ ]:
# save pickled dataset
with open("keyword_generation_dataset/generation_train_dataset_blenderbot.pickle", "wb") as f:
    pickle.dump(train_dataset, f)
with open("keyword_generation_dataset/generation_dev_dataset_blenderbot.pickle", "wb") as f:
    pickle.dump(validation_dataset, f)

Use pickled dataset

In [ ]:
# open pickled dataset
# with open("keyword_generation_dataset/generation_train_dataset_gpt2.pickle", "rb") as f:
#     train_dataset = pickle.load(f)
# with open("keyword_generation_dataset/generation_dev_dataset_gpt2.pickle", "rb") as f:
#     validation_dataset = pickle.load(f)

In [ ]:
batch_size = 8
train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size=batch_size, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_dataset, sampler = SequentialSampler(validation_dataset), batch_size=batch_size, collate_fn=collate_fn)

In [ ]:
num_epochs = 5
num_training_steps = len(train_dataloader) * num_epochs # Total number of training steps is [number of batches] x [number of epochs].
learning_rate = 2e-5 # 5e-4
warmup_ratio = 0.1
warmup_steps = int(warmup_ratio * num_training_steps ) # 1e2

# this produces sample output every 100 steps
sample_step = 100

# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                )

# lr_scheduler = get_linear_schedule_with_warmup(optimizer,
#                                             num_warmup_steps = warmup_steps,
#                                             num_training_steps = num_training_steps)

lr_scheduler = get_polynomial_decay_schedule_with_warmup(
                optimizer,
                num_warmup_steps=warmup_steps,
                num_training_steps=num_training_steps,
                power=2
                )

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using Cuda?: {torch.cuda.is_available()}")
model.to(device)

total_t0 = time.time()
progress_bar = tqdm(range(num_training_steps))
training_stats = []
best_loss = 100

for epoch in range(num_epochs):
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch + 1, num_epochs))

  # ========================================
  #               Training
  # ========================================
  print('Training...')
  t0 = time.time()
  total_train_loss = 0

  model.train()

  for step, batch in enumerate(train_dataloader):
    b_input_ids = batch['input_ids'].to(device)
    b_labels = batch['labels'].to(device)
    outputs = model(input_ids=b_input_ids,
                    labels=b_labels
                    )
    loss = outputs.loss
    total_train_loss += loss.item()

    # Get sample every x batches.
    if step % sample_step == 0 and not step == 0:
      elapsed = format_time(time.time() - t0)
      print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), loss, elapsed))

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()
    lr_scheduler.step()

    progress_bar.update(1)
  # Calculate the average loss over all of the batches.
  avg_train_loss = total_train_loss / len(train_dataloader)

  # Measure how long this epoch took.
  training_time = format_time(time.time() - t0)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epoch took: {:}".format(training_time))

  # ========================================
  #               Validation
  # ========================================

  print("")
  print("Running Validation...")

  t0 = time.time()

  model.eval()

  total_eval_loss = 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:

    b_input_ids = batch['input_ids'].to(device)
    b_labels = batch['labels'].to(device)
    outputs = model(input_ids=b_input_ids,
                    labels=b_labels
                    )
    loss = outputs.loss
    total_eval_loss += loss.item()

  avg_val_loss = total_eval_loss / len(validation_dataloader)

  validation_time = format_time(time.time() - t0)

  print("  Validation Loss: {0:.2f}".format(avg_val_loss))
  print("  Validation took: {:}".format(validation_time))

  # Record all statistics from this epoch.
  training_stats.append(
      {
          'epoch': epoch + 1,
          'Training Loss': avg_train_loss,
          'Valid. Loss': avg_val_loss,
          'Training Time': training_time,
          'Validation Time': validation_time
      }
  )

  # if avg_val_loss < best_loss:
  best_loss = avg_val_loss
  state_dict = {
                    'model_state_dict': model.state_dict(),
                    'optim_state_dict': optimizer.state_dict(),
                    'sched_state_dict': lr_scheduler.state_dict(),
                    'loss': best_loss,
                    'epoch': epoch + 1
                }

  torch.save(state_dict, f"blenderbot_best_ckpt_epoch={epoch+1}_valid_loss={round(best_loss, 4)}.ckpt")

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_blenderbot_more_data/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))


In [ ]:
import pandas as pd
# Display floats with two decimal places.
# pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
with open("model_blenderbot_more_data.pickle", "wb") as f:
  pickle.dump(training_stats, f)

In [ ]:
# Use plot styling from seaborn.
# sns.set(style='darkgrid')

# Increase the plot size and font size.
# sns.set(font_scale=1.5)
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4, 5])

plt.show()